# Lab 3: Nim Game

## Problem

- Deterministic
- Perfect Information
- Turn-based
- Zero Sum

 “If both A and B play optimally (i.e- they don’t make any mistakes), then the player starting first is guaranteed to win if the Nim-Sum at the beginning of the game is non-zero. Otherwise, if the Nim-Sum evaluates to zero, then player A will lose definitely.”
 https://en.wikipedia.org/wiki/Subtraction_game

## Library

In [2]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
import numpy as np
import operator as op

In [3]:
random.seed(42)

#  In class

## Nim Class

In [4]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [5]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k
        self._last_action = None

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k
    
    @property
    def last_action(self):
        return self._last_action
    
    def memory(self,last_action: Nimply):
        """
        Memorizing the last action
        """
        self._last_action = last_action


    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) strategies

In [6]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [7]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [8]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [9]:

def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [10]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

## Oversimplified match

In [11]:
NUM_MATCHES = 10
NIM_SIZE = 10


def evaluate(strategy: Callable) -> float:
    opponent = (strategy, optimal_startegy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

## Oversimplified match

In [12]:
logging.getLogger().setLevel(logging.DEBUG)

#strategy = (make_strategy({"p": 0.1}), optimal_startegy)
strategy = (optimal_startegy,optimal_startegy)
nim = Nim(11,5)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 17 21>
DEBUG:root:status: After player 1 -> <1 3 1 7 9 11 13 15 17 17 21>
DEBUG:root:status: After player 0 -> <1 2 1 7 9 11 13 15 17 17 21>
DEBUG:root:status: After player 1 -> <1 2 1 7 9 7 13 15 17 17 21>
DEBUG:root:status: After player 0 -> <1 2 1 7 9 7 13 15 13 17 21>
DEBUG:root:status: After player 1 -> <1 2 1 7 9 6 13 15 13 17 21>
DEBUG:root:status: After player 0 -> <0 2 1 7 9 6 13 15 13 17 21>
DEBUG:root:status: After player 1 -> <0 2 1 3 9 6 13 15 13 17 21>
DEBUG:root:status: After player 0 -> <0 2 1 3 9 2 13 15 13 17 21>
DEBUG:root:status: After player 1 -> <0 2 1 0 9 2 13 15 13 17 21>
DEBUG:root:status: After player 0 -> <0 1 1 0 9 2 13 15 13 17 21>
DEBUG:root:status: After player 1 -> <0 1 1 0 8 2 13 15 13 17 21>
DEBUG:root:status: After player 0 -> <0 0 1 0 8 2 13 15 13 17 21>
DEBUG:root:status: After player 1 -> <0 0 1 0 3 2 13 15 13 17 21>
DEBUG:

# The Game

In [13]:
class Player():
    
    def __init__(self,id,strategy = None):
        self._strategy = strategy
        self._id = id
    
    @property
    def strategy(self):
        return self._strategy
    
    @property
    def id(self):
        return self._id
    
    @property
    def id(self):
        return self._id
    
    def possible_moves(self,state: Nim):
        return [
            (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
        ]

In [14]:
class Game():
    """
    Game Simulation Class
    """
    def __init__(self,player1,player2,num_rows: int,k: int = None):
        #logging.getLogger().setLevel(logging.DEBUG)
        self._nim = Nim(num_rows,k)
        self._p1 = player1
        self._p2 = player2
        #logging.debug(f"status: Initial board  -> {self._nim}")
    
    @property
    def p1(self):
        return self._p1
    @property
    def p2(self):
        return self._p2   
    
    def run(self):
        player = self._p1
        while self._nim:
            ply = player.strategy(self._nim)
            self._nim.nimming(ply)
            #logging.debug(f"status: After player {player.id} -> {self._nim} removing {ply[1]}")
            if player == self._p1:
                player = self._p2
            else:
                player = self._p1
        if player == self._p1:
            winner = self._p2.id
        else:
            winner = self._p1.id
        #logging.info(f"status: Player {winner} won!")
        return  winner

## Task 3.1: Rule Based

In [15]:
class ExpertSystem(Player):
    """
    Expert System is type of player which apply the optimal strategy
    """
    def __init__(self,id):
        super().__init__(id)
    
    def nim_sum(self,state):
        *_, result = accumulate(state.rows, xor)
        return result

    def strategy(self,state):
        brute_force = list()
        for m in self.possible_moves(state):
            tmp = deepcopy(state)
            tmp.nimming(m)
            brute_force.append((m, self.nim_sum(tmp)))
        return next((bf for bf in brute_force if bf[1] == 0), random.choice(brute_force))[0]

In [16]:
# Testing the expert system class
p1 = Player("opponent",make_strategy({"p": 0.5}))
#p1 = ExpertSystem("expert_system1")
p2 = ExpertSystem("expert_system2")
game = Game(p1,p2, 11)
game.run()

'expert_system2'

In [17]:
class RuleBased(Player):
    def __init__(self,id, params = [0.2,0.16,0.16,0.16,0.16,0.16]):
        super().__init__(id)
        self._params = params
    
    @property
    def params(self):
        return self._params
    @params.setter
    def name(self, value):
        self._params = value
        
    def cooked_rules(self, state: Nim, moves: list):
        cooked = dict()
        moves_row = [el for el in state.rows if el != 0]
        
        # SIMPLE RULES
        simple = dict()
        shortest_row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0] #index shortest row (In class)
        longest_row =  max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
        simple["shortest_row"] = Nimply(shortest_row,self.choose_nim_quantity(state,state.rows[shortest_row]))
        simple["longest_row"] = Nimply(longest_row,self.choose_nim_quantity(state,state.rows[longest_row])) #index longest row (In class)
        simple["min_k"] = Nimply(state.rows.index(moves_row[random.randint(0,len(moves_row)-1)]),1) #nimming always 1
        #simple["same_action"] = state.last_action #try to play the same action as the opponent
        cooked["simple"] = simple
        
        #GENERALIZED RULES
        cooked["remaining_parity_row"] = self.action_strategy_row(state,cooked,moves_row)
        cooked["remaining_quantity_row"] = self.action_strategy_quantity(state,moves_row)
        cooked["reaction_longest_shortest"] = self.reaction_strategy_longest_vs_shortest(state,cooked)
        cooked["reaction_quantity"] = self.reaction_strategy_quantity(state,moves_row)
        cooked["random"] = self.pure_random(state)
        
        return cooked
    
    def choose_nim_quantity(self,state: Nim,row: int):
        if (state.k):
            if (row > state.k):
                return random.randint(1,state.k)   
        return random.randint(1,row)
        
    def clear_row(self,state: Nim, row_value: int):
        """action of clear a row"""
        return Nimply(state.rows.index(row_value),row_value)
    
    def  keep_one(self,state: Nim, row_value: int):
        """Try to keep the last one for the player"""
        return Nimply(state.rows.index(row_value),min(int(row_value- (state.k + 1)),state.k)) 
    
    def action_strategy_row(self, state: Nim, rules: dict, possible_rows: list):
        """
        According to the parity of the remaning rows
        """
        ply = None
        non_empty_row = possible_rows.copy()
        
        if (len(non_empty_row)%2 == 0):
            """If the number of rows remaining is pair we don't want to empty a row"""
            non_empty_row = sorted(non_empty_row, reverse = True) # It is harder to empty the longest one. 
            if state.k:
                if non_empty_row[0] < state.k:
                    """If the row size is less than the state.k, we only remove 1"""
                    ply = Nimply(state.rows.index(non_empty_row[0]),1)
                else:
                    ply = self.keep_one(state, non_empty_row[0])
            else:
                """If k is not defined we will try to force the opponent to take the last one"""
                longest = rules["simple"]["longest_row"][0]
                ply = Nimply(longest,state.rows[longest]-1)
    
        elif (len(non_empty_row)%2 != 0):
            """If the number of rows remaining is unpair we want to empty a row"""
            non_empty_row = sorted(non_empty_row) # It is easier to remove the shortest rows
            if state.k:
                if non_empty_row[0] <= state._k :
                    """The player can nim all pieces"""
                    ply = self.clear_row(state,non_empty_row[0])
                else:
                    """Try to keep the last one for the player and not the opponent"""
                    ply = self.keep_one(state,non_empty_row[0]) #Try to go to a quantity of state.k + 1 in the last row if not remove state.k
            else:
                ply = Nimply(rules["simple"]["shortest_row"][0], non_empty_row[0])
        return ply
    
    def action_strategy_quantity(self, state: nim,possible_rows: list):
        """
        Strategy according to the reamining quantity of nim
        """
        ply = None
        non_empty_row = possible_rows.copy()
        if(sum(state.rows)%2 ==0):
            """If the quantity remaining is paired we want to keep a paired quantity"""
            valid_rows  = [el for el in non_empty_row if el > 1] #> 1 because we want to nim a paired quantity
            if valid_rows:
                row = valid_rows[random.randint(0,len(valid_rows)-1)] # Select a random row where we can remove a pair number
                if state.k:  
                    if state.k == 1:
                        """This strategy focus on the quantity to nim so if state.k == 1 is the only case were we have only unpaired quantity to remove"""
                        nimming =[1]
                    else:
                        nimming = [el for el in range(1,state.k+1) if (el%2==0) & (el <= row)] #Nim only unpaired quantity possible in the selected row
                else:
                    if row == 1:
                        nimming = [1]
                    else:
                        nimming = [el for el in range(1,row+1) if (el%2==0) & (el <= row)] #Nim only unpaired quantity possible in the selected row
                ply = Nimply(state.rows.index(row), sorted(nimming,reverse=True)[0])
            else:
                row  = non_empty_row[random.randint(0,len(non_empty_row)-1)]#Select a random non empty row    
                if state.k:  
                    if state.k == 1:
                        """This strategy focus on the quantity to nim so if state.k == 1 is the only case were we have only unpaired quantity to remove"""
                        nimming =[1]
                    else:
                        nimming = [el for el in range(1,state.k+1) if (el%2!=0) & (el <= row)] #Nim only paired quantity possible in the selected row
                else:
                    if row == 1:
                        nimming = [1]
                    else:
                        nimming = [el for el in range(1,row+1) if (el%2!=0) & (el <= row)] #Nim only paired quantity possible in the selected row
                ply = Nimply(state.rows.index(row), sorted(nimming)[0])
        else:
            """If the quantity remainig is unpaired we want to get a paired quantity"""   
            row  = non_empty_row[random.randint(0,len(non_empty_row)-1)]#Select a random non empty row    
            if state.k:  
                if state.k == 1:
                    """This strategy focus on the quantity to nim so if state.k == 1 is the only case were we have only unpaired quantity to remove"""
                    nimming =[1]
                else:
                    nimming = [el for el in range(1,state.k+1) if (el%2!=0) & (el <= row)] #Nim only paired quantity possible in the selected row
            else:
                if row == 1:
                    nimming = [1]
                else:
                    nimming = [el for el in range(1,row+1) if (el%2!=0) & (el <= row)] #Nim only paired quantity possible in the selected row
            ply = Nimply(state.rows.index(row), sorted(nimming)[0])
        return ply
                 
    def reaction_strategy_longest_vs_shortest(self, state: Nim, rules:dict):
        """
        The player will play in a different domain than the opponent
        """
        if state.last_action:
            if state.row[state.last_action[0]] <= sum(state.rows)//2:
                """The oppponent nimmed shortest rows so I want to play on the longest"""
                row  = state.rows[rules["longest_row"]]#Select the longest row 
                if state.k:
                    if state.k == 1:
                         nimming =[1]
                    else:
                        nimming = [el for el in range(1,state.k+1) if el <= row] #Nim  quantity possible in the selected row
                else:
                    nimming = [el for el in range(1,row+1) if el <= row] #Nim quantity possible in the selected row
                ply = Nimply(state.rows.index(row), nimming[0])
            else:
                """The oppponent nimmed longest row so I want to play on the shortest"""
                row  = state.rows[rules["shortest_row"]]#Select the shortest row 
                if state.k:
                    if state.k == 1:
                         nimming =[1]
                    else:
                        nimming = [el for el in range(1,state.k+1) if el <= row] #Nim  quantity possible in the selected row
                else:
                    if row == 1:
                        nimming = [1]
                    else:
                        nimming = [el for el in range(1,row+1) if el <= row] #Noù quantity possible in the selected row
                ply = Nimply(state.rows.index(row), nimming[0])
            return ply
        else:
            return pure_random(state)
    
    def reaction_strategy_quantity(self, state: Nim, possible_rows):
        """
        The player will play different quantity than the oppponent
        """
        non_empty_row = possible_rows.copy()
        if state.last_action:
            row  = non_empty_row[random.randint(0,len(non_empty_row)-1)]#Select a random row 
            if state.k == 1:
                """This strategy focus on the quantity to nim so if state.k == 1 is the only case were we have only unpaired quantity to remove"""
                ply = Nimply(state.rows.index(row), 1)
            elif state.last_action[1]%2==0:
                """The oppponent nimmed shortest a pair quantity of nim"""
                if state.k:
                    nimming = [el for el in range(1,state.k+1) if (el%2!=0) & (el <= row)] #Nim quantity possible in the selected row
                else:
                    if row == 1:
                        nimming = [1]
                    else:
                        nimming = [el for el in range(1,row+1) if (el%2!=0) & (el <= row)] #Nim quantity possible in the selected row
                ply = Nimply(state.rows.index(row), nimming[0])
            else:
                """The oppponent nimmed shortest an unpair quantity of nim"""""
                if state.k:
                    nimming = [el for el in range(1,state.k+1) if (el%2==0) & (el <= row)] #Nim  quantity possible in the selected row
                else:
                    if row == 1:
                        nimming = [1]
                    else:
                        nimming = [el for el in range(1,row+1) if (el%2==0) & (el <= row)] #Noù quantity possible in the selected row
                ply = Nimply(state.rows.index(row), nimming[0])
            return ply
              
        else:
            return pure_random(state)
    
    def pure_random(self,state: Nim):
        row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
        num_objects = random.randint(1, state.rows[row])
        return Nimply(row, num_objects)
    
    def strategy(self, state:Nim): #DEFINE PARAMS AS ATTRIBUTE
        """
        strategies used to react to opponent actions, I xhoose to wait different kind of rules
        @state: the state of the nim game
        @params: probability of each strategy the simple strategy will be always the same. sum of probability all simple = 0.2
        """
        moves = self.possible_moves(state)
        rules = self.cooked_rules(state,moves)
        
        strategies_name = list(rules.keys())
        strategy_choosed = np.random.choice(strategies_name, p=self.params)
        if strategy_choosed == "simple":
            strategy_choosed = np.random.choice(list(rules[strategy_choosed].keys()))
            return rules["simple"][strategy_choosed]

        return rules[strategy_choosed]

In [18]:
NUM_MATCHES = 100
NIM_SIZE = 25

evolution = []
for i in range(NIM_SIZE):
    winner = []
    for m in range(NUM_MATCHES):
        nim = Nim(i)
        p1 = RuleBased("pure_random",[0,0,0,0,0,1])
        p2 = RuleBased("classic_rules")
        game = Game(p1,p2, i)
        winner.append(game.run())
    evolution.append(op.countOf(winner,"classic_rules")/len(winner))
    print(f'Percentage of victory against pure random:{op.countOf(winner,"classic_rules")/len(winner)} with nim size {i}')
np.mean(evolution)
    

Percentage of victory against pure random:1.0 with nim size 0
Percentage of victory against pure random:0.0 with nim size 1
Percentage of victory against pure random:0.57 with nim size 2
Percentage of victory against pure random:0.65 with nim size 3
Percentage of victory against pure random:0.63 with nim size 4
Percentage of victory against pure random:0.68 with nim size 5
Percentage of victory against pure random:0.65 with nim size 6
Percentage of victory against pure random:0.67 with nim size 7
Percentage of victory against pure random:0.66 with nim size 8
Percentage of victory against pure random:0.64 with nim size 9
Percentage of victory against pure random:0.64 with nim size 10
Percentage of victory against pure random:0.72 with nim size 11
Percentage of victory against pure random:0.62 with nim size 12
Percentage of victory against pure random:0.67 with nim size 13
Percentage of victory against pure random:0.57 with nim size 14
Percentage of victory against pure random:0.57 with 

0.6352

Therefore my strategy works better than pure random it will win with between 60% and 66% efficiency when random begin ()

In [19]:
NUM_MATCHES = 100
NIM_SIZE = 25

evolution = []
for i in range(NIM_SIZE):
    winner = []
    for m in range(NUM_MATCHES):
        nim = Nim(i)
        p2 = RuleBased("pure_random",[0,0,0,0,0,1])
        p1 = RuleBased("classic_rules")
        game = Game(p1,p2, i)
        winner.append(game.run())
    evolution.append(op.countOf(winner,"classic_rules")/len(winner))
    print(f'Percentage of victory against pure random:{op.countOf(winner,"classic_rules")/len(winner)} with nim size {i}')
np.mean(evolution)

Percentage of victory against pure random:0.0 with nim size 0
Percentage of victory against pure random:1.0 with nim size 1
Percentage of victory against pure random:0.65 with nim size 2
Percentage of victory against pure random:0.5 with nim size 3
Percentage of victory against pure random:0.56 with nim size 4
Percentage of victory against pure random:0.6 with nim size 5
Percentage of victory against pure random:0.63 with nim size 6
Percentage of victory against pure random:0.63 with nim size 7
Percentage of victory against pure random:0.67 with nim size 8
Percentage of victory against pure random:0.55 with nim size 9
Percentage of victory against pure random:0.66 with nim size 10
Percentage of victory against pure random:0.64 with nim size 11
Percentage of victory against pure random:0.63 with nim size 12
Percentage of victory against pure random:0.68 with nim size 13
Percentage of victory against pure random:0.67 with nim size 14
Percentage of victory against pure random:0.57 with ni

0.6164000000000001

In [20]:
NUM_MATCHES = 100
NIM_SIZE = 25

evolution = []
for i in range(NIM_SIZE):
    winner = []
    for m in range(NUM_MATCHES):
        nim = Nim(i)
        p2 = ExpertSystem("expert_system")
        p1 = RuleBased("classic_rules")
        game = Game(p1,p2, i)
        winner.append(game.run())
    evolution.append(op.countOf(winner,"classic_rules")/len(winner))
    print(f'Percentage of victory against expert_system:{op.countOf(winner,"classic_rules")/len(winner)} with nim size {i}')
np.mean(evolution)

Percentage of victory against expert_system:0.0 with nim size 0
Percentage of victory against expert_system:1.0 with nim size 1
Percentage of victory against expert_system:0.41 with nim size 2
Percentage of victory against expert_system:0.14 with nim size 3
Percentage of victory against expert_system:0.17 with nim size 4
Percentage of victory against expert_system:0.11 with nim size 5
Percentage of victory against expert_system:0.21 with nim size 6
Percentage of victory against expert_system:0.18 with nim size 7
Percentage of victory against expert_system:0.2 with nim size 8
Percentage of victory against expert_system:0.15 with nim size 9
Percentage of victory against expert_system:0.17 with nim size 10
Percentage of victory against expert_system:0.13 with nim size 11
Percentage of victory against expert_system:0.22 with nim size 12
Percentage of victory against expert_system:0.13 with nim size 13
Percentage of victory against expert_system:0.18 with nim size 14
Percentage of victory a

0.22640000000000002

In [21]:
NUM_MATCHES = 100
NIM_SIZE = 25

evolution = []
for i in range(NIM_SIZE):
    winner = []
    for m in range(NUM_MATCHES):
        nim = Nim(i)
        p1 = ExpertSystem("expert_system")
        p2 = RuleBased("classic_rules")
        game = Game(p1,p2, i)
        winner.append(game.run())
    evolution.append(op.countOf(winner,"classic_rules")/len(winner))
    print(f'Percentage of victory against expert_system:{op.countOf(winner,"classic_rules")/len(winner)} with nim size {i}')
np.mean(evolution)

Percentage of victory against expert_system:1.0 with nim size 0
Percentage of victory against expert_system:0.0 with nim size 1
Percentage of victory against expert_system:0.19 with nim size 2
Percentage of victory against expert_system:0.12 with nim size 3
Percentage of victory against expert_system:0.19 with nim size 4
Percentage of victory against expert_system:0.16 with nim size 5
Percentage of victory against expert_system:0.2 with nim size 6
Percentage of victory against expert_system:0.1 with nim size 7
Percentage of victory against expert_system:0.16 with nim size 8
Percentage of victory against expert_system:0.17 with nim size 9
Percentage of victory against expert_system:0.25 with nim size 10
Percentage of victory against expert_system:0.19 with nim size 11
Percentage of victory against expert_system:0.14 with nim size 12
Percentage of victory against expert_system:0.28 with nim size 13
Percentage of victory against expert_system:0.2 with nim size 14
Percentage of victory aga

0.22200000000000003

My strategy wins sometimes against expert_system

## Task 3.2: Evolved Rules

### Individuals

The genome will be define by an array of params like [0.2,0.16,0.16,0.16,0.16,0.16] where the sum is equal to 1

In [22]:
Individual = namedtuple("Individual", ["genome", "fitness"])

In [23]:
def generate_weights(weights: list):
    """Function that make the sum of weight == 1"""
    p = weights
    p   = np.array(p)
    p = p/p.sum()  # normalize
    return p

### Fitness

The fitness will be the average score during the evolution

In [24]:
NUM_MATCHES = 50
NIM_SIZE = 20
def fitness(genome: list):
    evolution = []
    for i in range(NIM_SIZE):
        winner = []
        for m in range(NUM_MATCHES):
            nim = Nim(i)
            p1 = RuleBased("classic_rules",genome)
            p2 = RuleBased("pure_random",[0,0,0,0,0,1])
            game = Game(p1,p2, i)
            winner.append(game.run())
        evolution.append(op.countOf(winner,"classic_rules")/len(winner))
        #print(f'Percentage of victory against pure random:{op.countOf(winner,"classic_rules")/len(winner)} with nim size {i}')
    return np.mean(evolution)

### Tournament

In [25]:
def tournament(population: list, tournament_size=2):
    """Tournament function"""
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

### Cross-Over

In [26]:
def cross_over(g1, g2, g3):
    cut = random.randint(0, 3)
    cut2 = random.randint(cut,5)
    return generate_weights(g1.tolist()[:cut] + g2.tolist()[cut:cut2] + g3.tolist()[cut2:])

In [27]:
a = [1,3,3,5,0,1]
b = [2,5,9,3,4,8]
a[:3]+b[3:]

[1, 3, 3, 3, 4, 8]

### Mutation

For mutation, we will permutate two parameters

In [28]:
def mutation(g:list):
    point1 = random.randint(0, len(g) - 1)
    point2 = None
    while not point2:
        test = random.randint(0, len(g) - 1)
        if test != point1:
            point2 = test
    tmp = g[point1]
    g[point1] = g[point2]
    g[point2] = tmp
    return g

### Evolution

#### Initial Population

In [29]:
def create_rand_pop(count):
    pop = []
    for i in range(count):
        logging.log
        genome = generate_weights([random.randint(1,100) for i in range(6)])
        logging.debug(f"status: Created {i} genome")
        fitness_value = fitness(genome)
        pop.append(Individual(genome,fitness_value))
    return pop

In [30]:
def evolution(population: list,POPULATION_SIZE: int, NUM_GENERATIONS:int, OFFSPRING_SIZE:int):
    for g in range(NUM_GENERATIONS):
        logging.debug(f"generation_number {g}")
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < 0.3:
                p = tournament(population)
                o = mutation(p.genome.copy())
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                p3 = tournament(population)
                o = cross_over(p1.genome.copy(), p2.genome.copy(),p3.genome.copy())
            f = fitness(o)
            offspring.append(Individual(o, f))
        population+=offspring
        population = sorted(population, key=lambda indi: indi.fitness,reverse=True)[:POPULATION_SIZE]
        logging.debug(f"top fitness {population[0].fitness}")
    #plot_result(fitness_log,10*N)
    return population[0]

In [33]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 10
NUM_GENERATIONS = 50
### Initialisation of the problem
population = create_rand_pop(POPULATION_SIZE)

DEBUG:root:status: Created 0 genome
DEBUG:root:status: Created 1 genome
DEBUG:root:status: Created 2 genome
DEBUG:root:status: Created 3 genome
DEBUG:root:status: Created 4 genome
DEBUG:root:status: Created 5 genome
DEBUG:root:status: Created 6 genome
DEBUG:root:status: Created 7 genome
DEBUG:root:status: Created 8 genome
DEBUG:root:status: Created 9 genome
DEBUG:root:status: Created 10 genome
DEBUG:root:status: Created 11 genome
DEBUG:root:status: Created 12 genome
DEBUG:root:status: Created 13 genome
DEBUG:root:status: Created 14 genome
DEBUG:root:status: Created 15 genome
DEBUG:root:status: Created 16 genome
DEBUG:root:status: Created 17 genome
DEBUG:root:status: Created 18 genome
DEBUG:root:status: Created 19 genome
DEBUG:root:status: Created 20 genome
DEBUG:root:status: Created 21 genome
DEBUG:root:status: Created 22 genome
DEBUG:root:status: Created 23 genome
DEBUG:root:status: Created 24 genome
DEBUG:root:status: Created 25 genome
DEBUG:root:status: Created 26 genome
DEBUG:root:

In [34]:
evolution(population,POPULATION_SIZE,NUM_GENERATIONS,OFFSPRING_SIZE)

DEBUG:root:generation_number 0
DEBUG:root:top fitness 0.7270000000000001
DEBUG:root:generation_number 1
DEBUG:root:top fitness 0.7309999999999999
DEBUG:root:generation_number 2
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 3
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 4
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 5
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 6
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 7
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 8
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 9
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 10
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 11
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 12
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 13
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 14
DEBUG:root:top fitness 0.769
DEBUG:root:generation_number 15
DEBUG:root:top fitness 0.769
DEBUG:ro

Individual(genome=array([0.04220121, 0.42820883, 0.40232793, 0.02478737, 0.04050624,
       0.06196842]), fitness=0.837)

### Best Result
Individual(genome=array([0.04220121, 0.42820883, 0.40232793, 0.02478737, 0.04050624,
       0.06196842]), fitness=0.837)